In [44]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.environ["MISTRAL_API_KEY"]

In [ ]:
from unstructured.partition.pdf import partition_pdf
from pathlib import Path
file_path = Path("..") / "docs" / "Manuel de formation - Telescope.pdf"

# Reference: https://docs.unstructured.io/open-source/core-functionality/chunking
elements = partition_pdf(
    filename=file_path,
    infer_table_structure=True,            # extract tables
    strategy="hi_res",                     # mandatory to infer tables
    extract_image_block_types=["Image"],   # Add 'Table' to list to extract image of tables
    # image_output_dir_path=output_path,   # if None, images and tables will be saved in base64

    extract_image_block_to_payload=True,   # if true, will extract base64 for API usage
)


In [46]:
core_elems = [
    el for el in elements
    if el.category not in ("Header", "Footer", "PageNumber")
]

In [47]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [48]:
text_model = ChatMistralAI(mistral_api_key=api_key,model="mistral-large-latest")
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.
The summary must be in french.
Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.
Optimize your summary embedding as it will be used for RAG.
Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)
summarize_chain = {"element": lambda x: x} | prompt | text_model | StrOutputParser()

In [49]:
def summarize_Table(item, summarize_chain):
    return summarize_chain.invoke({"element": item})


In [ ]:
from mistralai import Mistral

client = Mistral(api_key=api_key) 
image_summaries = []
def summarize_image(b64,prefix,suffix):
    prompt = f""" You are given:
- an inline image (base-64) extracted from a PDF
- the French text immediately before and after the image

Text-before:
{prefix}

Text-after:
{suffix}

Task:
- Write **one paragraph (≤ 150 words, in French)** that describes only what is visible in the image.
- If the image is the logo of "Association Gens de la lune", do not describe it.
- Use the surrounding text only to resolve names, labels or context; do not repeat or paraphrase it.
- Keep the prose concise and optimised for semantic retrieval (RAG).
- Your priority is to describe the image, not to summarize the text.
- Do not mention that you are describing an image, and do not start with phrases like “Cette image montre”.
- Do not include any additional information or context that is not visible in the image.
- If the image is not relevant to the text, do not describe it.
Respond in French.  Output only the paragraph."""
    prompt_template = prompt
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_template},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{b64}"}}
            ]
        }
    ]
    
    # Perform inference
    response = client.chat.complete(
        model="pixtral-large-latest",
        messages=messages,
    )
    # Return the model's output
    return response.choices[0].message.content


In [ ]:
def image_wrapper(text_list,i,max_chars=200,elements_nb=3):
    prefix = " \n".join(text_list[-elements_nb:])
    prefix = prefix[-max_chars:]
    suffix = [ el.text for el in core_elems[i:i+elements_nb] if el.category not in {"Image","Table"} ]
    suffix = " \n".join(suffix)
    if len(suffix) > max_chars:
        suffix = suffix[:max_chars]
    return prefix, suffix

In [55]:
text = ""
text_list = []
for i, element in enumerate(core_elems):
    if element.category == "Image":
        before_image = text_list[-3:]
        b64 = element.metadata.image_base64
        prefix, suffix = image_wrapper(text_list,i)
        image_summary = summarize_image(b64,prefix,suffix)
        print(image_summary + "\n")
        text+= image_summary + "\n"
    elif element.category == "Table":
        html = element.metadata.text_as_html
        table_summary = summarize_Table(html, summarize_chain)
        print(table_summary + "\n")
        text += table_summary + "\n"
        text_list.append(table_summary)
    else :
        print(element.text + "\n")
        text+= element.text  + "\n"
        text_list.append(element.text)
        

Un observatoire astronomique se dresse sous un ciel nocturne étoilé, teinté de nuances violettes et roses. Le bâtiment, de forme rectangulaire, possède un dôme blanc caractéristique, probablement abritant un télescope. Les environs sont sombres, avec des silhouettes d'arbres visibles en contrebas, suggérant un emplacement isolé, idéal pour l'observation des étoiles. L'horizon est légèrement illuminé par des teintes orangées, indiquant peut-être un coucher ou un lever de soleil.

Formation télescope OBSERVATOIRE DE LA POINTE DU DIABLE

Gens de la Lune | Formation | 2021

Un logo circulaire présentant un croissant de lune stylisé sur un fond blanc. Le texte « Gens de la Lune » est inscrit en lettres orange à côté. Le croissant de lune est épuré, avec une courbe élégante, symbolisant l'astronomie ou l'observation du ciel. Le design est minimaliste, mettant en avant la simplicité et l'élégance.

Table des matières

Le tableau présente divers aspects de l'instrumentation astronomique, inclu

KeyboardInterrupt: 